In [2]:
import matplotlib.pyplot as plt
import numpy as np
import time
import os
# from scipy.special import kv
# from scipy.integrate import quad
from importlib import reload
import pickle
from scipy.stats import maxwell

from pulsars import Pulsars, c, e_charge, e_mass, h
import speiser_fun as sf 
import speiser_fun_cyl as sfc
import speiser_integ as si
import speiser_plots as sp
import speiser_model as sm
import speiser_spec as ss

In [3]:
k = 500.
name = 'crab'
pulsar = Pulsars(k)[name]

gamma1 = 1.
B_0 = pulsar['Blc']
omegaB = (e_charge*B_0/(e_mass*c))    #γυροσυχνότητα
Rlc = pulsar['rlc']*omegaB/c
Delta = pulsar['rlc']/k*omegaB/c
# q = 1.

# gamma0 = maxwell.rvs(loc = 0, scale = 350, size=50)

# init_pos = np.linspace(0.01, 1., 30)

# delta = maxwell.stats(loc = 0, scale = 350, moments = 'm')    #πάχος του φύλλου ρεύματος, αδιάστατο, σε μονάδες [c/ωΒ]

delta_init = 500.
gamma0 = np.array([500., 1.])   #αρχικός παράγοντας Lorentz
init_pos = np.array([1.])


T = 500.
N = 5*10**6
t = np.linspace(0.0, T*Delta, N)


In [3]:
#Δημιουργία λεξικού με τις παραμέτους της προσωμοίωσης σε αναγνώσιμη και δυαδική μορφή
parameters = {'pulsar_name':name, 'char magn field (Blc)':B_0, 'Radius of lc':Rlc, 'multiplicity':k, 
              'diss zone length':Delta, 'diss zone width':delta_init, 'runtime':T, 'number of t points':N, 
              'initial lorentz factors':gamma0, 'initial injection positions':init_pos}

dir_name = 'data'

#Δημιουργεία φακέλου για την αποθηκευση των δεδομένων
for i in range(1, 1000):
    if not os.path.exists(dir_name):
        os.makedirs(dir_name)
        print('directory ', dir_name, 'created')
        break
    else:
        dir_name = 'data{}'.format(i)
        
with open('{}/parameters.txt'.format(dir_name), 'w') as par_file:
    for key, val in parameters.items():
        par_file.write('{} --> {}\n'.format(key, val))
with open('{}/par_dic.txt'.format(dir_name), 'wb') as par_file:
    pickle.dump(parameters, par_file)

directory  data11 created


In [4]:
for q in [-1]:
    if q == +1:
        charge = 'pos'
    else:
        charge = 'el'
    
    print(charge)
    
    for w in range(len(init_pos)):

        print(init_pos[w])

        (r, ur, phi, uphi, z_cyl, uz_cyl), dic_cyl = si.oloklirosi(gamma0, Rlc, Delta, delta_init, B_0, t, init_pos[w],
                                                                   q, T, sm.speiser_cyl, coord = 'cyl')
            
        with open('{}/cyl_{}_{}_raw.npz'.format(dir_name, int(100*init_pos[w]), charge), 'wb') as f:
            np.savez(f, r = r, ur = ur, phi = phi, uphi = uphi, z_cyl = z_cyl, uz_cyl = uz_cyl)

el
1.0


/home/petros/anaconda3/lib/python3.7/site-packages/scipy/integrate/odepack.py:236: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
  warnings.warn(warning_msg, ODEintWarning)


runtime for gamma0 = 500 is 0h 0m 44s
total runtime = 0h 0m 44s


In [4]:
dir_name = 'data10'
for q in [-1]:
    if q == +1:
        charge = 'pos'
    else:
        charge = 'el'
    for w in range(len(init_pos)):

        vec = np.load('{}/cyl_{}_{}_raw.npz'.format(dir_name, int(100*init_pos[w]), charge))
        r, ur, phi, uphi, z_cyl, uz_cyl = vec['r'], vec['ur'], vec['phi'], vec['uphi'], vec['z_cyl'], vec['uz_cyl']
        for i in range(len(gamma0) - 1):
            for j in range(len(r[i])):
                if r[i][j] < Rlc - 15*Delta or r[i][j] > Rlc + 15*Delta:
    #                     print(i, j, r[i][j])
                    r[i][j] = np.NaN
                    phi[i][j] = np.NaN
                    z_cyl[i][j] = np.NaN
                    ur[i][j] = np.NaN
                    uphi[i][j] = np.NaN
                    uz_cyl[i][j] = np.NaN
#                 if z_cyl[i][j] > 10000*sfc.delta(r[i][j], Rlc, Delta, delta_init) or z_cyl[i][j] < -10000*sfc.delta(r[i][j], Rlc, Delta, delta_init):
#                     r[i][j] = np.NaN
#                     phi[i][j] = np.NaN
#                     z_cyl[i][j] = np.NaN
#                     ur[i][j] = np.NaN
#                     uphi[i][j] = np.NaN
#                     uz_cyl[i][j] = np.NaN
        with open('{}/cyl_{}_{}.npz'.format(dir_name, int(100*init_pos[w]), charge), 'wb') as f:       
            np.savez(f, r = r, ur = ur, phi = phi, uphi = uphi, z_cyl = z_cyl, uz_cyl = uz_cyl)

In [7]:
#υπολογισμός ακτίνας καμπυλότητας, κρίσιμης συχνότητας και απωλειών

ss = reload(ss)
for q in [-1]:
    if q == +1:
        charge = 'pos'
    else:
        charge = 'el'
    
    for w in range(len(init_pos)):

        if not (w == 7):
            a = np.load('{}/cyl_{}_{}.npz'.format(dir_name, int(100*init_pos[w]), charge))
            r, ur, phi, uphi, z_cyl, uz_cyl = a['r'], a['ur'], a['phi'], a['uphi'], a['z_cyl'], a['uz_cyl']

            r_curv, nu_crit, p_rad = ss.rc_nc_pr(r, phi, z_cyl, ur, uphi, uz_cyl, Rlc, Delta, delta_init, gamma0, B_0)

            with open('{}/rc_nc_pr_{}_{}.npz'.format(dir_name, int(100*init_pos[w]), charge), 'wb') as f2:
                np.savez(f2, r_curv = r_curv, nu_crit = nu_crit,  p_rad = p_rad)

            en, ph_num, ph_num_out, ph_num_out2, ph_num_sep, ph_en, ph_en_out, ph_en_out2, ph_en_sep, ph_num_tot, ph_en_tot = ss.spectrum(r, 
                                                                                                                                          z_cyl, 
                                                                                                                                          nu_crit, 
                                                                                                                                          p_rad, 
                                                                                                                                          gamma0, 
                                                                                                                                          Rlc, 
                                                                                                                                          Delta, 
                                                                                                                                          delta_init, 
                                                                                                                                          t)

            with open('{}/spectrum_{}_{}.npz'.format(dir_name, int(100*init_pos[w]), charge), 'wb') as f3:
                np.savez(f3, en = en, ph_num = ph_num, ph_num_out = ph_num_out, ph_num_out2 = ph_num_out2, 
                         ph_num_sep = ph_num_sep, 
                         ph_en = ph_en, ph_en_out = ph_en_out, ph_en_out2 = ph_en_out2, ph_en_sep = ph_en_sep,
                         ph_num_tot = ph_num_tot, ph_en_tot = ph_en_tot)